In [15]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [1]:
# --- ورود به حساب کاربری Hugging Face ---
# این کد را پس از تنظیم 'HF_TOKEN' در Kaggle Secrets، در یک سلول جداگانه اجرا کنید.
from huggingface_hub import login
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("HF_TOKEN")
try:
    login(token=secret_value)
    print("\nSuccessfully logged in to Hugging Face using Kaggle Secret.")
except:
    print("\nError: HF_TOKEN Kaggle Secret not found. Please ensure it's created and attached.")
    print("You can try manual login by uncommenting the line below and running again.")
    # login() # uncomment this line to try manual login if secret method fails



Successfully logged in to Hugging Face using Kaggle Secret.


In [2]:
from datasets import load_dataset

print("Loading the gaokerena/MF3QA dataset from Hugging Face Hub...")
try:
    dataset = load_dataset("gaokerena/MF3QA")
    print("Dataset loaded successfully.")
    print("\nDataset structure:")
    print(dataset)
    
    if 'train' in dataset:
        train_dataset = dataset['train']
    else:
        train_dataset = dataset[list(dataset.keys())[0]]

    print(f"\nNumber of examples in the training split: {len(train_dataset)}")
    print("\nFirst 5 raw examples from the dataset:")
    for i in range(min(5, len(train_dataset))):
        print(f"--- Example {i+1} ---")
        # *** اصلاح شده: استفاده از 'Question' و 'Answer' با حرف بزرگ ***
        print(f"Question: {train_dataset[i]['Question']}")
        print(f"Answer: {train_dataset[i]['Answer']}")

    # --- فرمت بندی دیتاست برای Instruction Tuning ---
    def format_example(example):
        # *** اصلاح شده: استفاده از 'Question' و 'Answer' با حرف بزرگ ***
        question = str(example.get('Question', '')).strip()
        answer = str(example.get('Answer', '')).strip()

        formatted_text = f"سوال: {question}\nپاسخ: {answer}"
        return {"text": formatted_text}

    print("\nFormatting the dataset into 'text' column...")
    # remove_columns باید با نام ستون های اصلی دیتاست مطابقت داشته باشد.
    formatted_dataset = train_dataset.map(format_example, remove_columns=train_dataset.column_names)
    
    print("\nFirst 3 formatted examples:")
    for i in range(min(3, len(formatted_dataset))):
        print(f"--- Formatted Example {i+1} ---")
        print(formatted_dataset[i]['text'])

    print("\nDataset preparation for fine-tuning is complete. Ready for model loading and tokenization.")

except Exception as e:
    print(f"An error occurred during dataset loading or preparation: {e}")
    print("Please ensure you have successfully logged in to Hugging Face and your internet connection is stable.")

Loading the gaokerena/MF3QA dataset from Hugging Face Hub...


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/796k [00:00<?, ?B/s]

...

Generating dev split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset loaded successfully.

Dataset structure:
DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer', 'Source'],
        num_rows: 20000
    })
    dev: Dataset({
        features: ['Question', 'Answer', 'Source'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['Question', 'Answer', 'Source'],
        num_rows: 2000
    })
})

Number of examples in the training split: 20000

First 5 raw examples from the dataset:
--- Example 1 ---
Question: ۱ماهپیش از خواب پریدم از ترس شدید بعداز اون ترس،فشارم بالا میرفت ۱۴ ۱۵ با سر درد و درد قفسه سینه همراه بود با پرانول کنترش میکردم ولی الان ۲ ۳ روزه فشارم میاد پاین ۸ ۹ روی ۳ ۴ وقتایی که میرم پیاده روی یا کلا فعالیتی دارم بعدش ضربانم تا ۶ ۷ ساعت حتی بیشتر تنده و کند و طبیعی نمیشه خستهه شدم دیگهمیترسم اتفاقی برام بیوفته فشار ۸ ۹ روری ۴ ۵ خطرناکه؟
Answer: تجربه‌ای که شرح داده‌اید، نشان‌دهنده‌ی رخدادهای فیزیولوژیک و احتمالاً پاتولوژیک در بدن شما است. ترس شدید و پرش از خواب می‌تواند منجر به افزایش موقت فشار خو

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]


First 3 formatted examples:
--- Formatted Example 1 ---
سوال: ۱ماهپیش از خواب پریدم از ترس شدید بعداز اون ترس،فشارم بالا میرفت ۱۴ ۱۵ با سر درد و درد قفسه سینه همراه بود با پرانول کنترش میکردم ولی الان ۲ ۳ روزه فشارم میاد پاین ۸ ۹ روی ۳ ۴ وقتایی که میرم پیاده روی یا کلا فعالیتی دارم بعدش ضربانم تا ۶ ۷ ساعت حتی بیشتر تنده و کند و طبیعی نمیشه خستهه شدم دیگهمیترسم اتفاقی برام بیوفته فشار ۸ ۹ روری ۴ ۵ خطرناکه؟
پاسخ: تجربه‌ای که شرح داده‌اید، نشان‌دهنده‌ی رخدادهای فیزیولوژیک و احتمالاً پاتولوژیک در بدن شما است. ترس شدید و پرش از خواب می‌تواند منجر به افزایش موقت فشار خون و تپش قلب شود، که این واکنشی طبیعی است. با این حال، ادامه‌ی این علائم و تغییرات فشار خون از بالا به پایین نیازمند بررسی بیشتر است. فشار خون شما که گاهی به 8/9 می‌رسد و ضربان قلب بالا پس از فعالیت، ممکن است نشانه‌ای از اختلال در تنظیم فشار خون یا مشکلات قلبی باشد. مراجعه به پزشک، انجام آزمایش‌های تخصصی قلب و عروق و شاید مشاوره با یک روانپزشک یا روانشناس برای مدیریت استرس و ترس شدید پیشنهاد می‌شود. اطمینان از کنترل درست فشار 

In [9]:
print("Ensuring all necessary libraries are installed...")
!pip install -q -U transformers peft trl bitsandbytes scipy datasets
!pip install -q -U "huggingface_hub[cli]"
# !git config --global user.name "lbehradl"
# !git clone https://github.com/unslothai/unsloth.git
# !pip install -q -U ./unscdloth
# !pip install -q -U unsloth_zoo
print("Libraries installation/update complete.")


Ensuring all necessary libraries are installed...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.7.8 requires datasets<4.0.0,>=3.4.1, but you have datasets 4.0.0 which is incompatible.
Libraries installation/update complete.


In [12]:
!pip install -U "scipy==1.11.4"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 50.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.0
    Uninstalling scipy-1.16.0:
      Successfully uninstalled scipy-1.16.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.11.4 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompat

In [16]:
import unsloth
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
try:
    from unsloth import FastLanguageModel
    print("Unsloth detected. Using FastLanguageModel for optimized loading.")
    USE_UNSLOTH = True
except ImportError:
    print("Unsloth not found. Falling back to standard Hugging Face loading.")
    USE_UNSLOTH = False
print('imported')




Unsloth detected. Using FastLanguageModel for optimized loading.
imported


In [17]:
model_name = "google/medgemma-4b-it" # یا "google/medgemma-27b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", # نوع کوانتیزیشن
    bnb_4bit_compute_dtype=torch.bfloat16, # نوع داده برای محاسبات
    bnb_4bit_use_double_quant=True, # استفاده از کوانتیزیشن دوگانه برای کاهش بیشتر حافظه
)

# --- بارگذاری مدل و توکنایزر ---
# اگر Unsloth در دسترس باشد، از آن برای بارگذاری بهینه استفاده می‌کنیم.
if USE_UNSLOTH:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_name,
        max_seq_length=1024, # حداکثر طول دنباله (sequence length) برای ورودی های مدل
                               # این مقدار را می توانید بر اساس طول سوالات و پاسخ های خود تنظیم کنید.
                               # 2048 یک مقدار رایج و مناسب برای شروع است.
        dtype=None, # None به unsloth اجازه می دهد بهترین dtype را انتخاب کند (معمولا bfloat16)
        load_in_4bit=True, # فعال کردن کوانتیزیشن 4-bit
    )
else:
    # بارگذاری توکنایزر
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # اطمینان از تنظیم pad_token برای توکنایزر
    # این برای مدل های decoder-only مهم است.
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token # End-of-sequence token به عنوان pad_token

    # بارگذاری مدل با تنظیمات کوانتیزیشن
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto", # برای توزیع مدل روی GPU های موجود
        torch_dtype=torch.bfloat16, # استفاده از bfloat16 برای محاسبات
    )
    # آماده سازی مدل برای آموزش LoRA با کوانتیزیشن 4-bit
    model = prepare_model_for_kbit_training(model)

print(f"\nModel '{model_name}' and Tokenizer loaded successfully.")
print("\nModel structure (first few layers):")
print(model) # نمایش ساختار مدل

# --- تنظیمات توکنایزر برای آموزش ---
# این تنظیمات برای اطمینان از اینکه توکنایزر به درستی برای آموزش آماده است، ضروری است.
tokenizer.padding_side = "right" # پدینگ از سمت راست (برای مدل های decoder-only توصیه می شود)

print("\nTokenizer padding side set to 'right'.")
print("Ready for LoRA configuration and training.")

==((====))==  Unsloth 2025.7.6: Fast Gemma3 patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.12G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]


Model 'google/medgemma-4b-it' and Tokenizer loaded successfully.

Model structure (first few layers):
Gemma3ForConditionalGeneration(
  (model): Gemma3Model(
    (vision_tower): SiglipVisionModel(
      (vision_model): SiglipVisionTransformer(
        (embeddings): SiglipVisionEmbeddings(
          (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
          (position_embedding): Embedding(4096, 1152)
        )
        (encoder): SiglipEncoder(
          (layers): ModuleList(
            (0-15): 16 x SiglipEncoderLayer(
              (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
              (self_attn): SiglipAttention(
                (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (out_proj): Linear(in_features=1152, out_feature

In [20]:
# --- مرحله 4: پیکربندی LoRA و Trainer ---
# این کد را در یک سلول جدید در Kaggle Notebook خودتان اجرا کنید.

# مطمئن شوید که 'model' و 'tokenizer' از مرحله قبل در دسترس هستند.
# همچنین 'formatted_dataset' که در مرحله 2 آماده کردیم.

from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments
from trl import SFTTrainer # SFTConfig در اینجا استفاده نمی شود، از TrainingArguments استفاده می کنیم.

print("Configuring LoRA and Training Arguments for GPU...")

# --- 4.1: پیکربندی LoRA (Low-Rank Adaptation) ---
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# --- اعمال پیکربندی LoRA به مدل ---
# اگر از Unsloth استفاده می کنید، FastLanguageModel.from_pretrained ممکن است این کار را به صورت خودکار انجام دهد.
# اگر خطای "Adapter with name default already exists" گرفتید، این خط را کامنت کنید.
model = get_peft_model(model, lora_config)

print("LoRA adapters attached to the model.")

# --- 4.2: تنظیمات آموزش (TrainingArguments) ---
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2, # تعداد اپوک ها برای آموزش.
    per_device_train_batch_size=4, # *** اصلاح شده: افزایش بچ سایز به 4 برای استفاده بیشتر از GPU ***
    gradient_accumulation_steps=2, # *** اصلاح شده: کاهش گام های انباشت گرادیان برای حفظ effective batch size (4 * 2 = 8) ***
    optim="paged_adamw_8bit", # بهینه ساز برای QLoRA در GPU.
    save_steps=500, # تعداد گام ها برای ذخیره چک پوینت مدل.
    logging_steps=50, # تعداد گام ها برای لاگ کردن اطلاعات آموزش (loss و غیره).
    learning_rate=2e-4, # نرخ یادگیری برای بهینه ساز.
    fp16=False, # استفاده از float32 (با تنظیم هر دو fp16 و bf16 به False).
    bf16=False, # استفاده از float32 (با تنظیم هر دو fp16 و bf16 به False).
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    report_to="tensorboard",
    disable_tqdm=False, # فعال کردن نوار پیشرفت برای مشاهده وضعیت.
    torch_compile=False, # غیرفعال کردن torch.compile برای سازگاری با P100.
    # اگر می خواهید مدل را در Hugging Face Hub آپلود کنید، این پارامترها را فعال کنید:
    # push_to_hub=True,
    # hub_model_id="your-username/medgemma-fa-medical-qa",
    # hub_private_repo=False,
    # hub_strategy="every_save",
)

# --- 4.3: آماده سازی SFTTrainer ---
trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    peft_config=lora_config,
    args=training_arguments,
    tokenizer=tokenizer,
    max_seq_length=512, # طول دنباله برای سرعت بیشتر.
    dataset_text_field="text",
    packing=False,
)

print("\nLoRA and Training Arguments configured successfully.")
print("SFTTrainer initialized. Ready to start training!")


Configuring LoRA and Training Arguments for GPU...


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


LoRA adapters attached to the model.
Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"]:   0%|          | 0/20000 [00:00<?, ? examples/s]


LoRA and Training Arguments configured successfully.
SFTTrainer initialized. Ready to start training!


In [19]:
# --- مرحله 5: اجرای فاین تیونینگ ---
# این کد را در یک سلول جدید در Kaggle Notebook خودتان اجرا کنید.

# مطمئن شوید که 'trainer' از مرحله 4 در دسترس است.

print("Starting the fine-tuning process. This may take a while depending on your GPU and dataset size.")
print("The training progress will be logged based on 'logging_steps' defined in TrainingArguments.")

# شروع فرایند آموزش
trainer.train()

print("\nFine-tuning process completed successfully!")

# --- مرحله 6: ذخیره مدل فاین تیون شده ---
# پس از اتمام آموزش، مدل فاین تیون شده را ذخیره می کنیم.
# این شامل وزن های LoRA (آداپتورها) است که می تواند با مدل پایه ادغام شود.

# مسیر ذخیره سازی مدل
# این مسیر باید با output_dir در TrainingArguments مطابقت داشته باشد.
output_dir_model = "./results/final_model"

print(f"\nSaving the fine-tuned model to: {output_dir_model}")
# ذخیره آداپتورهای LoRA
trainer.model.save_pretrained(output_dir_model)
print("LoRA adapters saved.")

# ذخیره توکنایزر
tokenizer.save_pretrained(output_dir_model)
print("Tokenizer saved.")

# --- ادغام آداپتورهای LoRA با مدل پایه (اختیاری و برای استقرار نهایی) ---
# این بخش نیاز به RAM GPU بیشتری دارد. اگر در آینده قصد استقرار مدل کامل را دارید،
# می توانید این بخش را فعال کنید. فعلاً کامنت شده است.
# اگر از Unsloth استفاده می کنید، می توانید از تابع FastLanguageModel.save_pretrained_merged استفاده کنید.
# if USE_UNSLOTH:
#     print("\nMerging LoRA adapters with the base model and saving the merged model...")
#     # save_method="merged_16bit" یا "merged_4bit" بسته به نیاز شما
#     trainer.model.save_pretrained_merged(output_dir_model, tokenizer, save_method = "merged_4bit")
#     print("Merged model saved.")
# else:
#     print("\nTo merge LoRA adapters with the base model (without Unsloth), run the following (requires more RAM):")
#     print("from peft import PeftModel")
#     print("from transformers import AutoModelForCausalLM, AutoTokenizer")
#     print(f"base_model_loaded = AutoModelForCausalLM.from_pretrained('{model_name}', torch_dtype=torch.float32, device_map='auto')")
#     print(f"merged_model = PeftModel.from_pretrained(base_model_loaded, '{output_dir_model}')")
#     print("merged_model = merged_model.merge_and_unload()")
#     print("merged_model.save_pretrained('./results/merged_model')")
#     print("tokenizer.save_pretrained('./results/merged_model')")


print("\nModel fine-tuning and saving process complete. You can now load this model for inference.")


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,000 | Num Epochs = 3 | Total steps = 3,750
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 32,788,480 of 4,332,867,952 (0.76% trained)


Starting the fine-tuning process. This may take a while depending on your GPU and dataset size.
The training progress will be logged based on 'logging_steps' defined in TrainingArguments.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss


KeyboardInterrupt: 